导入依赖

In [54]:
import pandas as pd
from util.helpers import my_txt_reader
from util.DataProcessor import LabelRepository
from scipy import stats

读取DeepFix变异生成的训练集数据，并统计个数

In [55]:
# 准备标签字典
labels = LabelRepository()
labels.map_get()
labels.label_map_reverse

{0: 'IDNoDef',
 1: 'BoundLost',
 2: 'NotInclude',
 3: 'BoundNotMatch',
 4: 'BoundWrongUsage',
 5: 'opDataLost',
 6: 'BoundRedundant',
 7: 'APIcallWrongUsage',
 8: 'KeyWordWrongUsage',
 9: 'opWrongUsage',
 10: 'TypeWrongUsage',
 11: 'IDWrongUsage',
 12: 'opLost'}

In [56]:
# 读取数据
mutate_classified = r'data/deepfix_classify_result.txt'
mutate_class_dict_mutate = my_txt_reader(mutate_classified)

In [57]:
mutate_classified = r'data/deepfix_classify_result_ids.txt'
mutate_class_dict_ids = my_txt_reader(mutate_classified)

In [58]:
print(len(mutate_class_dict_mutate))
print(len(mutate_class_dict_ids))

283790
216321


In [59]:
time_all = len(mutate_class_dict_mutate) + len(mutate_class_dict_ids)
mutate_class_dict = {}

time_counter = 0
for class_value in mutate_class_dict_mutate.values():
    mutate_class_dict[time_counter] = class_value
    time_counter += 1
for class_value in mutate_class_dict_ids.values():
    mutate_class_dict[time_counter] = class_value
    time_counter += 1

In [60]:
len(mutate_class_dict)

500111

In [61]:
mutate_dict = {}
for error_class_id in list(set(mutate_class_dict.values())):
    mutate_dict[int(error_class_id)] = 0

for code_id, error_class_id in mutate_class_dict.items():
    mutate_dict[int(error_class_id)] += 1

del mutate_dict[-1]

获取带有修复结果的数据，并统计个数

In [62]:
result_deepfixS = pd.read_excel(r"data/ALL_fix_state.xlsx", sheet_name="deepfixS_fix_state")
result_grouped_deepfixS = result_deepfixS.groupby("error_class_id")
result_grouped_deepfixS.count()

result_dict = {}
for error_class_id, result_df in result_grouped_deepfixS:
    result_dict[error_class_id] = result_df.shape[0]

In [63]:
#用kolmogorov-Smirnov检验deepfix和iitk是否同分布
mutate_class = list(mutate_class_dict.values())
result_deepfixS = pd.read_excel(r"data/ALL_fix_state.xlsx", sheet_name="deepfixS_fix_state")

print(stats.kstest(result_deepfixS["error_class_id"], mutate_class))

KstestResult(statistic=0.2784558893217802, pvalue=7.783684837493369e-199)


In [64]:
# 卡方优度拟合检验
statistic_df = pd.DataFrame({"true":result_dict,"mutate":mutate_dict})

statistic_df["true_pro"] = statistic_df["true"]/statistic_df["true"].sum()
statistic_df["mutate_pro"] = statistic_df["mutate"]/statistic_df["mutate"].sum()

n_sample = statistic_df["mutate"].sum()

statistic_df["true_resample"] = n_sample * statistic_df["true_pro"]
statistic_df["mutate_resample"] = statistic_df["mutate"]
# statistic_df["mutate_resample"] = n_sample * statistic_df["mutate_pro"]

# statistic_df["true_resample"] = statistic_df["true_resample"].astype('int64')
# statistic_df["mutate_resample"] = statistic_df["mutate_resample"].astype('int64')
print(stats.chisquare(
        f_obs=statistic_df["mutate_resample"],
        f_exp=statistic_df["true_resample"]
    )
)

print(stats.chisquare(
        f_obs=[10,10,10,10,10],
        f_exp=[10,10,10,11,9]
    )
)

Power_divergenceResult(statistic=1800998.1739312282, pvalue=0.0)
Power_divergenceResult(statistic=0.20202020202020202, pvalue=0.9952293469715736)


In [65]:
print(statistic_df["true"].sum())
print(statistic_df["mutate"].sum())

2910
457762


比较deepfixS和iitk的分布差异

In [66]:
result_deepfixS = pd.read_excel(r"data/ALL_fix_state.xlsx", sheet_name="deepfixS_fix_state")
result_iitk = pd.read_excel(r"data/ALL_fix_state.xlsx", sheet_name="ittk_fix_state")

def result_dict_get(result_df):
    result_df_grouped = result_df.groupby("error_class_id")
    result_df_grouped.count()

    result_dict = {}
    for error_class_id, result_df_id in result_df_grouped:
        result_dict[error_class_id] = result_df_id.shape[0]

    return result_dict

result_deepfixS_dict = result_dict_get(result_deepfixS)
result_iitk_dict = result_dict_get(result_iitk)

In [67]:
# 卡方优度拟合检验
statistic_df = pd.DataFrame({"deepfixS":result_deepfixS_dict,"iitk":result_iitk_dict})

statistic_df["deepfixS_pro"] = statistic_df["deepfixS"]/statistic_df["deepfixS"].sum()
statistic_df["iitk_pro"] = statistic_df["iitk"]/statistic_df["iitk"].sum()

n_sample = statistic_df["iitk"].sum()
# n_sample = 100000

statistic_df["deepfixS_resample"] = n_sample * statistic_df["deepfixS_pro"]
# statistic_df["iitk_resample"] = statistic_df["iitk"]
statistic_df["iitk_resample"] = n_sample * statistic_df["iitk_pro"]

# statistic_df["deepfixS_resample"] = statistic_df["true_resample"].astype('int64')
# statistic_df["iitk_resample"] = statistic_df["mutate_resample"].astype('int64')
print(stats.chisquare(
        f_obs=statistic_df["iitk_resample"],
        f_exp=statistic_df["deepfixS_resample"]
    )
)

print(stats.chisquare(
        f_obs=[10,10,10,10,10],
        f_exp=[10,10,10,11,9]
    )
)

Power_divergenceResult(statistic=4448.039518371882, pvalue=0.0)
Power_divergenceResult(statistic=0.20202020202020202, pvalue=0.9952293469715736)


In [68]:
statistic_df["deepfixS"].sum()
statistic_df["iitk"].sum()

14015